### The RIDB API
<img src="images/hobbit_house_expedia.jpeg"style="display: inline-block">
<br>
image source: expedia.com
<br>
<br>
For an RIDB API Key: https://ridb.recreation.gov/?action=register

In [75]:
import pandas as pd 
import numpy as np
import config
import requests
import json
from pandas.io.json import json_normalize

#### Get the data from RDIB

In [62]:
ridb_facilities_url = config.RIDB_ENDPOINT + "/facilities"
camping_params = params=dict(activity_id=9, state='OR', apiKey = config.RIDB_API_KEY,\
                             latitude=45.3300, longitude=-121.7089, radius=5)
response = requests.get(ridb_facilities_url,camping_params)

In [ ]:
mock_url = "http://" + config.LAMP_IP + "/ridb_mthood_camping_mock.json"
response = pd.read_json(mock_url)

In [63]:
camping_json  = json.loads(response.text)
camping_df = json_normalize(camping_json['RECDATA'])

In [64]:
camping_df.head()

,FacilityAdaAccess,FacilityDescription,FacilityDirections,FacilityEmail,FacilityID,FacilityLatitude,FacilityLongitude,FacilityMapURL,FacilityName,FacilityPhone,FacilityReservationURL,FacilityTypeDescription,FacilityUseFeeDescription,GEOJSON.COORDINATES,GEOJSON.TYPE,Keywords,LastUpdatedDate,LegacyFacilityID,OrgFacilityID,StayLimit
0,,<h2>Overview</h2>Trillium Lake Campground is a...,"From Portland, travel southeast on Highway 26 ...",,232831,45.270000,-121.734444,,TRILLIUM,503-272-3220,,Camping,,"[-121.73444444444445, 45.269999999999996]",Point,,2016-05-26,71614,AN371614,
1,,<h2>Overview</h2>Still Creek Campground lies i...,"From Portland, travel east on Highway 26 to Go...",,232835,45.295833,-121.735556,,STILL CREEK,541-328-0909,,Camping,,"[-121.73555555555555, 45.295833333333334]",Point,,2016-05-26,71618,AN371618,
2,,<p>Dispersed camping is available from this tr...,"<p><u>From Hood River, OR</u>&nbsp;travel 36.2...",,235865,45.282100,-121.678700,,Devils Half Acre Trailhead,,,,,"[-121.6787, 45.2821]",Point,,2016-05-09,,53072,
3,,<p>The Hidden Lake Trailhead leads to the Hidd...,"<p>From <u>Portland, OR</u> - follow U.S. Hwy ...",,235880,45.313528,-121.799667,,Hidden Lake Trailhead,,,,,"[-121.799667, 45.313528]",Point,,2016-05-09,,53128,
4,,<p>This trail and Sno Park is along the Histor...,"<p>From <u>Hood River, OR</u> travel 36.2 mile...",,235883,45.282969,-121.684789,,Barlow Pass Sno Park/Trailhead,,,,,"[-121.684789, 45.282969]",Point,,2016-05-09,,53138,


In [68]:
camping_df.LastUpdatedDate.unique()

array(['2016-05-26', '2016-05-09', '2015-09-18'], dtype=object)

In [73]:
camping_df.LegacyFacilityID.unique()

array([71614.0, 71618.0, ''], dtype=object)

#### Clean up the data

In [99]:
camping_df = camping_df.replace('', np.nan)

Drop geojson coordinates - Lat/Long information duplicated, mysql 5.4 doesnt support geojson types (but 5.7 does!)

In [100]:
camping_df = camping_df.drop(['GEOJSON.COORDINATES','GEOJSON.TYPE'], axis=1)

In [131]:
sum(camping_df.FacilityAdaAccess.isnull())

29

#### Store the data

In [130]:
camping_df.to_csv('test.csv', index=False)

In [106]:
csv_test = pd.read_csv('test.csv')

In [107]:
csv_test.head()

,FacilityAdaAccess,FacilityDescription,FacilityDirections,FacilityEmail,FacilityID,FacilityLatitude,FacilityLongitude,FacilityMapURL,FacilityName,FacilityPhone,FacilityReservationURL,FacilityTypeDescription,FacilityUseFeeDescription,Keywords,LastUpdatedDate,LegacyFacilityID,OrgFacilityID,StayLimit
0,NaN,<h2>Overview</h2>Trillium Lake Campground is a...,"From Portland, travel southeast on Highway 26 ...",NaN,232831,45.270000,-121.734444,NaN,TRILLIUM,503-272-3220,NaN,Camping,NaN,NaN,2016-05-26,71614.0,AN371614,NaN
1,NaN,<h2>Overview</h2>Still Creek Campground lies i...,"From Portland, travel east on Highway 26 to Go...",NaN,232835,45.295833,-121.735556,NaN,STILL CREEK,541-328-0909,NaN,Camping,NaN,NaN,2016-05-26,71618.0,AN371618,NaN
2,NaN,<p>Dispersed camping is available from this tr...,"<p><u>From Hood River, OR</u>&nbsp;travel 36.2...",NaN,235865,45.282100,-121.678700,NaN,Devils Half Acre Trailhead,NaN,NaN,NaN,NaN,NaN,2016-05-09,NaN,53072.0,NaN
3,NaN,<p>The Hidden Lake Trailhead leads to the Hidd...,"<p>From <u>Portland, OR</u> - follow U.S. Hwy ...",NaN,235880,45.313528,-121.799667,NaN,Hidden Lake Trailhead,NaN,NaN,NaN,NaN,NaN,2016-05-09,NaN,53128.0,NaN
4,NaN,<p>This trail and Sno Park is along the Histor...,"<p>From <u>Hood River, OR</u> travel 36.2 mile...",NaN,235883,45.282969,-121.684789,NaN,Barlow Pass Sno Park/Trailhead,NaN,NaN,NaN,NaN,NaN,2016-05-09,NaN,53138.0,NaN


In [109]:
from sqlalchemy import create_engine
connectStr = "mysql+pymysql://" + config.DB_USER + ":" + config.DB_PASS + "@" + config.DB_HOST +  "/" + config.DB_NAME
engine =create_engine(connectStr)

In [116]:
camping_df.to_sql('test',engine,if_exists='replace')

In [121]:
sql_test = pd.read_sql('select * from test', engine, index_col='index')

In [126]:
sql_test.head()

,FacilityAdaAccess,FacilityDescription,FacilityDirections,FacilityEmail,FacilityID,FacilityLatitude,FacilityLongitude,FacilityMapURL,FacilityName,FacilityPhone,FacilityReservationURL,FacilityTypeDescription,FacilityUseFeeDescription,Keywords,LastUpdatedDate,LegacyFacilityID,OrgFacilityID,StayLimit
index,,,,,,,,,,,,,,,,,,
0,None,<h2>Overview</h2>Trillium Lake Campground is a...,"From Portland, travel southeast on Highway 26 ...",None,232831,45.270000,-121.734444,None,TRILLIUM,503-272-3220,None,Camping,None,None,2016-05-26,71614.0,AN371614,None
1,None,<h2>Overview</h2>Still Creek Campground lies i...,"From Portland, travel east on Highway 26 to Go...",None,232835,45.295833,-121.735556,None,STILL CREEK,541-328-0909,None,Camping,None,None,2016-05-26,71618.0,AN371618,None
2,None,<p>Dispersed camping is available from this tr...,"<p><u>From Hood River, OR</u>&nbsp;travel 36.2...",None,235865,45.282100,-121.678700,None,Devils Half Acre Trailhead,None,None,None,None,None,2016-05-09,NaN,53072,None
3,None,<p>The Hidden Lake Trailhead leads to the Hidd...,"<p>From <u>Portland, OR</u> - follow U.S. Hwy ...",None,235880,45.313528,-121.799667,None,Hidden Lake Trailhead,None,None,None,None,None,2016-05-09,NaN,53128,None
4,None,<p>This trail and Sno Park is along the Histor...,"<p>From <u>Hood River, OR</u> travel 36.2 mile...",None,235883,45.282969,-121.684789,None,Barlow Pass Sno Park/Trailhead,None,None,None,None,None,2016-05-09,NaN,53138,None


In [143]:
sql_test.LastUpdatedDate.unique()

array(['2016-05-26', '2016-05-09', '2015-09-18'], dtype=object)

In [144]:
sum(sql_test.LastUpdatedDate.isnull())

0

In [146]:
sql_test.dropna(subset=['LastUpdatedDate'], axis=0).shape

(29, 18)

In [147]:
csv_test.dropna(subset=['LastUpdatedDate'], axis=0).shape

(29, 18)

In [149]:
camping_df.dropna(subset=['LastUpdatedDate'], axis=0).shape

(29, 18)